In [2]:
import json
import pandas as pd
import numpy as np
import re
from gensim.corpora.dictionary import Dictionary
import operator
import matplotlib.pyplot as plt

In [3]:
pwd

'/Users/zhangyilun/Desktop/Module2'

In [111]:
#这一个part是在处理商家数据
##读入数据
file3 = open('./rawdata/business_train.json','r')
busTrain = file3.readlines()
file3.close()
for i in range(len(busTrain)):
    busTrain[i]=json.loads(busTrain[i])
dfBusTrain = pd.DataFrame.from_records(busTrain)

,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,state
0,{'GoodForKids': 'False'},3157,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,AZ
1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",23732,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,NC
2,None,53580,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,AZ
3,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",27909,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,NC
4,"{'RestaurantsPriceRange2': '2', 'BusinessParki...",95694,"Beauty & Spas, Hair Salons",Calgary,None,1,50.943646,-114.001828,Edgeworxx Studio,T2Z 1K4,AB
5,"{'RestaurantsPriceRange2': '3', 'GoodForKids':...",72533,"Hair Salons, Hair Stylists, Barbers, Men's Hai...",Las Vegas,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",1,36.099872,-115.074574,Supercuts,89121,NV
6,"{'RestaurantsPriceRange2': '2', 'ByAppointment...",5438,"Nail Salons, Beauty & Spas, Day Spas",Glendale,"{'Tuesday': '12:0-18:0', 'Wednesday': '10:0-18...",0,33.654815,-112.188568,Vita Bella Fine Day Spa,85308,AZ


In [130]:
dfBusTrain.head(7).iloc[6]['hours']

{'Friday': '10:0-17:0',
 'Saturday': '10:0-14:0',
 'Thursday': '12:0-18:0',
 'Tuesday': '12:0-18:0',
 'Wednesday': '10:0-18:0'}

# 1.Find the chinese restaurant

In [19]:
def checkChinese(self):
    result = False
    try:
        catList = self.lower().replace(" ", "").split(',')
        if 'chinese'in catList:
            result=True
    except AttributeError:
        result = False
    return result
chineseIndex = np.where(dfBusTrain['categories'].apply(checkChinese))
chineseIdList = dfBusTrain['business_id'].iloc[chineseIndex]
dfChineseTrain = dfBusTrain.iloc[chineseIndex]
dfChineseTrain.index = range(len(dfChineseTrain))
dfChineseTrain.drop(['is_open', 'postal_code'],axis=1,inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
chineseIndex

(array([    38,     63,    149, ..., 154574, 154576, 154595]),)

In [29]:
chineseIdList.head()

38     105372
63      20037
149    111668
179    101172
236    165961
Name: business_id, dtype: int64

In [24]:
len(dfChineseTrain)

3557

In [110]:
dfChineseTrain.head(6)

,attributes,business_id,categories,city,hours,latitude,longitude,name,state
0,"{'BusinessParking': '{'garage': False, 'street...",105372,"Chinese, Dim Sum, Restaurants",Calgary,None,51.081162,-113.993474,Pearl Garden,AB
1,"{'Caters': 'True', 'GoodForKids': 'True', 'Noi...",20037,"Chinese, Restaurants",Phoenix,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",33.613020,-111.977036,Nee House Chinese Restaurant,AZ
2,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...",111668,"Chinese, Restaurants",Scottsdale,"{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3...",33.478754,-111.925484,Yummy Yummy Chinese Restaurant,AZ
3,"{'WiFi': ''no'', 'GoodForKids': 'True', 'Resta...",101172,"Chinese, Restaurants",Mesa,None,33.394863,-111.756036,China Gourmet,AZ
4,"{'Alcohol': ''full_bar'', 'RestaurantsTableSer...",165961,"Dim Sum, Chinese, Tapas/Small Plates, Restaurants",Toronto,"{'Monday': '11:30-22:30', 'Tuesday': '11:30-22...",43.646478,-79.390019,Pearl King,ON
5,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",138910,"Local Flavor, Chinese, Nightlife, Restaurants,...",Montréal,"{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...",45.567943,-73.552666,Jardin Tiki,QC


In [30]:
dfChineseTrain['categories'].head(5)

0                        Chinese, Dim Sum, Restaurants
1                                 Chinese, Restaurants
2                                 Chinese, Restaurants
3                                 Chinese, Restaurants
4    Dim Sum, Chinese, Tapas/Small Plates, Restaurants
Name: categories, dtype: object

# 2.对餐厅种类进行提取

In [39]:
tst1 = dfChineseTrain['categories'][0]
tst1

'Chinese, Dim Sum, Restaurants'

In [44]:
cat = re.sub('\s+|\)','',tst1.lower())    #将空格去除，且小写化
cat

'chinese,dimsum,restaurants'

In [45]:
cat = re.sub('&|/|\(',',',cat).split(',')
cat
           

['chinese', 'dimsum', 'restaurants']

In [51]:
chineseCatListFrame = []
def addChineseCat(self):
    cat = re.sub('\s+|\)','',self.lower())    #
    cat = list(set(re.sub('&|/|\(',',',cat).split(',')) - set(['restaurants', 'restaurant', 'food','chinese']))
    chineseCatListFrame.extend(cat)
dfChineseTrain['categories'].apply(addChineseCat)
chineseCatListFrame[:5]

['dimsum', 'dimsum', 'smallplates', 'tapas', 'buffets']

In [46]:
###和其他类型的餐厅相比，chinese的第二标签比较少，针对性更强
pd.Series(chineseCatListFrame).value_counts()[:20]

asianfusion      475
dimsum           242
thai             240
fastfood         229
specialtyfood    221
japanese         209
ethnicfood       196
buffets          195
seafood          182
sushibars        168
noodles          168
vietnamese       114
soup              95
cantonese         87
nightlife         85
szechuan          81
barbeque          81
taiwanese         79
importedfood      77
bars              74
dtype: int64

# 3.对餐厅属性进行提取

In [58]:
chineseAttrKeyList = []
def getChineseKeys(self):
    try:
        chineseAttrKeyList.extend(self.keys())
    except AttributeError:
        next
dfChineseTrain['attributes'].apply(getChineseKeys)
pd.Series(chineseAttrKeyList).value_counts()
validChineseAttrs = pd.Series(chineseAttrKeyList).value_counts().index[:18]
chineseAttrKeyList[:5]


['BusinessParking',
 'RestaurantsGoodForGroups',
 'OutdoorSeating',
 'RestaurantsPriceRange2',
 'GoodForKids']

In [49]:
validChineseAttrs

Index(['RestaurantsTakeOut', 'RestaurantsPriceRange2',
       'RestaurantsGoodForGroups', 'GoodForKids', 'RestaurantsDelivery',
       'RestaurantsReservations', 'RestaurantsAttire', 'OutdoorSeating',
       'BusinessParking', 'HasTV', 'Ambience', 'Alcohol', 'NoiseLevel', 'WiFi',
       'BikeParking', 'Caters', 'GoodForMeal', 'BusinessAcceptsCreditCards'],
      dtype='object')

In [145]:
validChineseAttrs.shape

(18,)

# 4.将餐厅属性制成dataframe

In [52]:
chineseAttrListFrame = []
def sub_dict(dct, keys, default=None):
    return dict([(key, dct.get(key, default)) for key in keys])
def addChineseAttr(self):
    try:
        validKeys = list(set(list(self.keys())).intersection(set(validChineseAttrs)))
        validSubDict = sub_dict(self, validKeys)
        chineseAttrListFrame.append(validSubDict)
    except AttributeError:
        chineseAttrListFrame.append({})
dfChineseTrain['attributes'].apply(addChineseAttr)
chineseAttrListFrame[0]

{'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
 'GoodForKids': 'True',
 'OutdoorSeating': 'False',
 'RestaurantsAttire': "u'casual'",
 'RestaurantsDelivery': 'False',
 'RestaurantsGoodForGroups': 'True',
 'RestaurantsPriceRange2': '1',
 'RestaurantsTakeOut': 'True'}

In [53]:
dfchineseAttrFrame = pd.DataFrame.from_records(chineseAttrListFrame)
dfchineseAttrFrame.head(5)

,Alcohol,Ambience,BikeParking,BusinessAcceptsCreditCards,BusinessParking,Caters,GoodForKids,GoodForMeal,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTakeOut,WiFi
0,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,True,NaN,NaN,NaN,False,u'casual',False,True,1,NaN,True,NaN
1,'beer_and_wine',"{'romantic': False, 'intimate': False, 'touris...",True,True,"{'garage': False, 'street': False, 'validated'...",True,True,"{'dessert': False, 'latenight': False, 'lunch'...",False,u'average',False,'casual',False,True,2,True,True,u'no'
2,u'none',"{'romantic': False, 'intimate': False, 'touris...",True,True,"{'garage': False, 'street': False, 'validated'...",True,True,NaN,True,'average',False,u'casual',True,False,1,False,True,u'no'
3,NaN,NaN,False,True,"{'garage': False, 'street': False, 'validated'...",NaN,True,NaN,NaN,u'quiet',False,'casual',True,True,1,False,True,'no'
4,'full_bar',"{'romantic': False, 'intimate': False, 'classy...",True,NaN,"{'garage': False, 'street': True, 'validated':...",True,True,"{'dessert': False, 'latenight': False, 'lunch'...",True,'average',True,'casual',False,True,2,True,True,u'free'


# 5.逐属性检索各餐厅是否有,并返回value

In [54]:
def getAttrs(x, attr):
    try:
        return x.get(attr, '')
    except AttributeError:
        return ''                      #检索是否有错误

In [63]:
takeoutList = [getAttrs(x, 'RestaurantsTakeOut') for x in chineseAttrListFrame]
takeoutList[:40]

['True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 '',
 'True',
 'True',
 'True',
 'True',
 'True']

In [64]:
pd.Series(takeoutList).value_counts()

True     3142
          316
False      97
None        2
dtype: int64

In [65]:
takeoutList = [getAttrs(x, 'RestaurantsTakeOut') for x in chineseAttrListFrame]
###空白表示有5000+家店是没有takeout记录的，None表示有45家店takeout记录显示为None？？？
pd.Series(takeoutList).value_counts()
priceList = [getAttrs(x, 'RestaurantsPriceRange2') for x in chineseAttrListFrame]
pd.Series(priceList).value_counts()
good4groupList = [getAttrs(x, 'RestaurantsGoodForGroups') for x in chineseAttrListFrame]
pd.Series(good4groupList).value_counts()
good4kidsList = [getAttrs(x, 'GoodForKids') for x in chineseAttrListFrame]
pd.Series(good4kidsList).value_counts()
reserveList = [getAttrs(x, 'RestaurantsReservations') for x in chineseAttrListFrame]
pd.Series(reserveList).value_counts()
deliverList = [getAttrs(x, 'RestaurantsDelivery') for x in chineseAttrListFrame]
pd.Series(deliverList).value_counts()
outdoorseatList = [getAttrs(x, 'OutdoorSeating') for x in chineseAttrListFrame]
pd.Series(outdoorseatList).value_counts()
tvList = [getAttrs(x, 'HasTV') for x in chineseAttrListFrame]
pd.Series(tvList).value_counts()
bikeparkList = [getAttrs(x, 'BikeParking') for x in chineseAttrListFrame]
pd.Series(bikeparkList).value_counts()
catersList = [getAttrs(x, 'Caters') for x in chineseAttrListFrame]
pd.Series(catersList).value_counts()
cardsList = [getAttrs(x, 'BusinessAcceptsCreditCards') for x in chineseAttrListFrame]
pd.Series(cardsList).value_counts()

         1777
True     1730
False      47
None        3
dtype: int64

In [67]:
###下列这几个是有大小写及元字符的问题，修改一下就好，值得关注的是应该分几级
def removeUnicode(self):
    try:
        return re.sub('u|\'','',self)
    except TypeError:
        return None

clothList = [getAttrs(x, 'RestaurantsAttire') for x in chineseAttrListFrame]
pd.Series(clothList).value_counts()
dfchineseAttrFrame['RestaurantsAttire'] = dfchineseAttrFrame['RestaurantsAttire'].apply(removeUnicode)
alcoholList = [getAttrs(x, 'Alcohol') for x in chineseAttrListFrame]
pd.Series(alcoholList).value_counts()
dfchineseAttrFrame['Alcohol'] = dfchineseAttrFrame['Alcohol'].apply(removeUnicode)
noiseList = [getAttrs(x, 'NoiseLevel') for x in chineseAttrListFrame]
pd.Series(noiseList).value_counts()
dfchineseAttrFrame['NoiseLevel'] = dfchineseAttrFrame['NoiseLevel'].apply(removeUnicode)
wifiList = [getAttrs(x, 'WiFi') for x in chineseAttrListFrame]
pd.Series(wifiList).value_counts()
dfchineseAttrFrame['WiFi'] = dfchineseAttrFrame['WiFi'].apply(removeUnicode)

carparkList = [getAttrs(x, 'BusinessParking') for x in chineseAttrListFrame]
pd.Series(carparkList).value_counts()
ambienceList = [getAttrs(x, 'Ambience') for x in chineseAttrListFrame]
pd.Series(ambienceList).value_counts()
good4mealList = [getAttrs(x, 'GoodForMeal') for x in chineseAttrListFrame]
pd.Series(good4mealList).value_counts().head()

                                                                                                                1760
{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}       761
{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}     439
{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}      219
{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'brunch': False, 'breakfast': False}      109
dtype: int64

## 处理属性中的parking

In [70]:
##处理park数据
def carCan(x):
    result = ''
    if x[1] == True:
        result = x[0]
    return result

def carFriend(self):
    try:
        canList = list(set([carCan(x) for x in list(eval(self).items())]) - set(['']))
        if len(canList)>0:
            if len(set(canList) - set(['lot','street']))>0:
                return 2
            else:
                return 1
        else:
            return 0
    except AttributeError:
        return None
    except SyntaxError:
        return None
###得到carFriendList
carFriendList = list(map(carFriend, carparkList))
pd.Series(carFriendList).value_counts()
dfchineseAttrFrame['BusinessParking'] = carFriendList
carFriendList[:10]

[0, 1, 1, 0, 1, None, 1, 1, 1, 0]

# 6.处理hours

In [71]:
##处理hours
dfChineseTrain['hours'].head(5)

0                                                 None
1    {'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...
2    {'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3...
3                                                 None
4    {'Monday': '11:30-22:30', 'Tuesday': '11:30-22...
Name: hours, dtype: object

In [72]:

###天数？
daysList = []
def checkDays(self):
    try:
        daysList.append(len(self.keys()))
    except AttributeError:
        daysList.append(None)
dfChineseTrain['hours'].apply(checkDays)
###可以发现大部分都是5天以上的，我认为在这里依据天数进行划分是没有意义的
pd.Series(daysList).value_counts()



7.0    2033
6.0     456
5.0      42
1.0       3
2.0       3
3.0       2
4.0       1
dtype: int64

In [74]:
###时间段？
hoursList = []
def checkHours(self):
    try:
        hoursList.append(list(self.values()))
    except AttributeError:
        hoursList.append(None)
dfChineseTrain['hours'].apply(checkHours)[:10]

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: hours, dtype: object

In [136]:
hoursList[:7]

[None,
 ['11:0-21:0',
  '11:0-21:0',
  '11:0-21:0',
  '11:0-21:0',
  '11:0-21:30',
  '11:0-21:30',
  '11:0-21:30'],
 ['11:0-21:30',
  '11:0-21:30',
  '11:0-21:30',
  '11:0-21:30',
  '11:0-22:0',
  '11:0-22:0',
  '11:0-21:30'],
 None,
 ['11:30-22:30',
  '11:30-22:30',
  '11:30-22:30',
  '11:30-22:30',
  '11:30-22:30',
  '11:30-22:30',
  '11:30-22:30'],
 ['11:30-21:0',
  '11:30-21:0',
  '11:30-21:0',
  '11:30-21:0',
  '11:30-23:0',
  '11:30-23:0',
  '11:30-21:0'],
 ['0:0-0:0',
  '11:0-19:0',
  '11:0-21:30',
  '11:0-21:30',
  '11:0-22:0',
  '11:0-22:0',
  '11:0-21:0']]

In [87]:
tst2 = hoursList[2]
openTime = pd.to_datetime(tst2[0].split('-')[0], format="%H:%M")
tst2[0].split('-')

['11:0', '21:30']

In [88]:
closeTime = pd.to_datetime(tst2[0].split('-')[1], format="%H:%M")
closeTime

Timestamp('1900-01-01 21:30:00')

In [89]:
(closeTime - openTime).seconds/3600

10.5

In [90]:
###计算时间段长度，这里只取第一天的长度
def calcuHours(self):
    try:
        openTime = pd.to_datetime(self[0].split('-')[0], format="%H:%M")
        closeTime = pd.to_datetime(self[0].split('-')[1], format="%H:%M")
        return (closeTime - openTime).seconds/3600
    except TypeError:
        return None

hoursRange = list(map(calcuHours, hoursList))
hoursRange[:5]

[None, 10.0, 10.5, None, 11.0]

In [109]:
pd.Series(hoursRange).value_counts().head(6)   #注意到营业0时的饭店有125家

11.0    537
10.5    361
10.0    358
12.0    224
11.5    179
0.0     125
dtype: int64

In [135]:
pd.Series(hoursRange)[:7]    #index 6

0     NaN
1    10.0
2    10.5
3     NaN
4    11.0
5     9.5
6     0.0
dtype: float64

In [137]:
hoursList[6]    #问题来了

['0:0-0:0',
 '11:0-19:0',
 '11:0-21:30',
 '11:0-21:30',
 '11:0-22:0',
 '11:0-22:0',
 '11:0-21:0']

In [94]:
pd.Series(hoursRange).describe().head()


count    2540.000000
mean       10.049278
std         3.306043
min         0.000000
25%        10.000000
dtype: float64

In [95]:
###这里最好画个图吧，我的air不行了
###大于15小时，昼夜餐厅
len((np.where(pd.Series(hoursRange)>=15))[0])


92

In [96]:
###小于6小时，特殊营业餐厅
len((np.where(pd.Series(hoursRange)<=6))[0])

357

In [138]:
def cateHours(self):
    type = 'all_day'
    try:
        openTime = pd.to_datetime(self[0].split('-')[0], format="%H:%M")
        closeTime = pd.to_datetime(self[0].split('-')[1], format="%H:%M")
        range = (closeTime - openTime).seconds/3600
        if range > 15:
            type = '24_hours'
        if range < 6:
            if closeTime < pd.to_datetime('15:00', format="%H:%M"):
                type = 'brunch'
            else:
                type = 'dinner'
    except TypeError:
        type = None
    return type
hoursCate = list(map(cateHours, hoursList))
pd.Series(hoursCate).value_counts()
dfChineseTrain['hours'] = hoursCate

In [141]:
hoursCate[:5]

[None, 'all_day', 'all_day', None, 'all_day']

# 7.合并上述改动

In [142]:
##最后得到处理过后的dfchineseAttrFrame，删掉了'Ambience', 'GoodForMeal'，同时与删掉了'attributes', 'categories','latitude','longitude'
##的dfChineseTrain进行合并
dfchineseAttrFrameMerge = dfchineseAttrFrame.drop(['Ambience', 'GoodForMeal'],axis=1)   
dfChineseTrainMerge = dfChineseTrain.drop(['attributes', 'categories','latitude','longitude'],axis=1)
dfChineseBusMerge = pd.concat([dfChineseTrainMerge, dfchineseAttrFrameMerge], axis=1)

In [143]:
dfChineseBusMerge.head()

,business_id,city,hours,name,state,Alcohol,BikeParking,BusinessAcceptsCreditCards,BusinessParking,Caters,...,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTakeOut,WiFi
0,105372,Calgary,None,Pearl Garden,AB,None,NaN,NaN,0.0,NaN,...,NaN,None,False,casal,False,True,1,NaN,True,None
1,20037,Phoenix,all_day,Nee House Chinese Restaurant,AZ,beer_and_wine,True,True,1.0,True,...,False,average,False,casal,False,True,2,True,True,no
2,111668,Scottsdale,all_day,Yummy Yummy Chinese Restaurant,AZ,none,True,True,1.0,True,...,True,average,False,casal,True,False,1,False,True,no
3,101172,Mesa,None,China Gourmet,AZ,None,False,True,0.0,NaN,...,NaN,qiet,False,casal,True,True,1,False,True,no
4,165961,Toronto,all_day,Pearl King,ON,fll_bar,True,NaN,1.0,True,...,True,average,True,casal,False,True,2,True,True,free


In [144]:
dfChineseBusMerge.shape    #21 = 18-2+5(5包括id,city,hours,name,state)

(3557, 21)

# 8.考虑review


In [148]:
# chinese_review part

file = open('./rawdata/review_train.json','r')
j2 = file.readlines()
for i in range(len(j2)):
    j2[i]=json.loads(j2[i])
dfReviewTrain = pd.DataFrame.from_records(j2)



In [149]:
dfReviewTrain.head()

,business_id,date,stars,text
0,31292,2013-05-07 04:34:36,1.0,Total bill for this horrible service? Over $8G...
1,35344,2017-01-14 21:30:33,5.0,I *adore* Travis at the Hard Rock's new Kelly ...
2,152538,2016-11-09 20:09:03,5.0,I have to say that this office really has it t...
3,71871,2018-01-09 20:56:38,5.0,Went in for a lunch. Steak sandwich was delici...
4,64913,2018-01-30 23:07:38,1.0,Today was my second out of three sessions I ha...


# 我电脑卡了= =

In [ ]:
##得到chinese restaurants的评论
chineseIdList = list(chineseIdList)
def getChinese(self):
    if self in chineseIdList:
        return True
    else:
        return False
### !!!为什么这一行会奇慢无比？？？
chineseReviewIndex = np.where(dfReviewTrain['business_id'].apply(getChinese))
len(chineseReviewIndex[0])
dfChineseReviewTrain = dfReviewTrain.iloc[resReviewIndex]
dfChineseReviewTrain.index = range(len(dfResReviewTrain))
dfChineseReviewTrain.head(5)


##准备与business数据合并
dfChineseReviewTrain = dfChineseReviewTrain.loc[:,['business_id','stars','text']]
dfChineseFinalMerge = pd.merge(dfChineseBusMerge, dfChineseReviewTrain, on='business_id')
len(dfChineseFinalMerge)
dfChineseFinalMerge.head(5)
dfChineseFinalMerge.to_csv('/Applications/Study/UWM/628/module2/chinesereview.csv')